# Data Collection and Pre-Processing Lab Assignment

## Step 1: Hello, Data!

Load the raw CSV and display the first 3 rows.

In [6]:
import pandas as pd

# Load first 500 rows from your CSV (adjust path as needed)
df = pd.read_csv('100000 Sales Records.csv').head(500)
df.head(3)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Azerbaijan,Snacks,Online,C,10/8/2014,535113847,10/23/2014,934,152.58,97.44,142509.72,91008.96,51500.76
1,Central America and the Caribbean,Panama,Cosmetics,Offline,L,2/22/2015,874708545,2/27/2015,4551,437.20,263.33,1989697.20,1198414.83,791282.37
2,Sub-Saharan Africa,Sao Tome and Principe,Fruits,Offline,M,12/9/2015,854349935,1/18/2016,9986,9.33,6.92,93169.38,69103.12,24066.26


## Step 2: Pick the Right Container

For transaction records, a Python class is most flexible: it allows attribute access and methods for cleaning and calculations. While `dict` or `namedtuple` can store data, only a class can encapsulate both data and behavior (e.g., `.clean()`, `.total()`).

In [7]:
from typing import Optional

class Transaction:
    def __init__(self, region, country, item_type, sales_channel, order_priority,
                 order_date, order_id, ship_date, units_sold, unit_price, unit_cost,
                 total_revenue, total_cost, total_profit):
        self.region = region
        self.country = country
        self.item_type = item_type
        self.sales_channel = sales_channel
        self.order_priority = order_priority
        self.order_date = order_date
        self.order_id = order_id
        self.ship_date = ship_date
        self.units_sold = float(units_sold)
        self.unit_price = float(unit_price)
        self.unit_cost = float(unit_cost)
        self.total_revenue = float(total_revenue)
        self.total_cost = float(total_cost)
        self.total_profit = float(total_profit)

    def clean(self):
        # Example cleaning: fix negative units or prices
        if self.units_sold < 0:
            self.units_sold = abs(self.units_sold)
        if self.unit_price < 0:
            self.unit_price = abs(self.unit_price)

    def total(self):
        # Calculate total revenue
        return self.units_sold * self.unit_price

## Step 4: Bulk Loader

Define a function to load transactions from the DataFrame.

In [13]:
from typing import List

def load_transactions(df: pd.DataFrame) -> List[Transaction]:
    transactions = []
    for _, row in df.iterrows():
        t = Transaction(
            row['Region'], row['Country'], row['Item Type'], row['Sales Channel'],
            row['Order Priority'], row['Order Date'], row['Order ID'],
            row['Ship Date'], row['Units Sold'], row['Unit Price'],
            row['Unit Cost'], row['Total Revenue'], row['Total Cost'],
            row['Total Profit']
        )
        transactions.append(t)
    return transactions

transactions = load_transactions(df)
print(f"Loaded {len(transactions)} transactions.")

Loaded 500 transactions.


## Step 5: Quick Profiling

Show min, mean, max unit price, and unique shipping countries.

In [14]:
unit_prices = [t.unit_price for t in transactions]
print("Min unit price:", min(unit_prices))
print("Mean unit price:", sum(unit_prices)/len(unit_prices))
print("Max unit price:", max(unit_prices))

unique_countries = set(t.country for t in transactions)
print("Unique countries:", len(unique_countries))

Min unit price: 9.33
Mean unit price: 278.65074
Max unit price: 668.27
Unique countries: 173


## Step 6: Spot the Grime

Identify at least three dirty data cases:
- Missing or NaN values in key fields
- Negative or zero units sold
- Inconsistent country or region names

In [15]:
import numpy as np

# Example checks
missing_units = sum(pd.isna(t.units_sold) for t in transactions)
negative_units = sum(t.units_sold <= 0 for t in transactions)
missing_country = sum(pd.isna(t.country) or t.country.strip() == '' for t in transactions)

print("Missing units sold:", missing_units)
print("Negative or zero units sold:", negative_units)
print("Missing country:", missing_country)

Missing units sold: 0
Negative or zero units sold: 0
Missing country: 0


## Step 7: Cleaning Rules

Apply fixes inside `.clean()` and show before/after counts.

In [16]:
# Before cleaning
neg_before = sum(t.units_sold < 0 for t in transactions)

# Apply cleaning
for t in transactions:
    t.clean()

neg_after = sum(t.units_sold < 0 for t in transactions)
print(f"Negative units before: {neg_before}, after cleaning: {neg_after}")

Negative units before: 0, after cleaning: 0


## Step 8: Transformations

Example: Standardize region names (title case), parse order dates.

In [17]:
from datetime import datetime

for t in transactions:
    t.region = t.region.title()
    try:
        t.order_date = datetime.strptime(str(t.order_date), '%m/%d/%Y')
    except Exception:
        pass  # Keep as string if parsing fails

## Step 9: Feature Engineering

Add days between order and ship date.

In [18]:
for t in transactions:
    try:
        order_dt = datetime.strptime(str(t.order_date), '%m/%d/%Y')
        ship_dt = datetime.strptime(str(t.ship_date), '%m/%d/%Y')
        t.days_to_ship = (ship_dt - order_dt).days
    except Exception:
        t.days_to_ship = None

# Add to DataFrame for later serialization
df['days_to_ship'] = [t.days_to_ship for t in transactions]

## Step 10: Mini-Aggregation

Revenue per country.

In [19]:
country_revenue = {}
for t in transactions:
    country_revenue[t.country] = country_revenue.get(t.country, 0) + t.total()

import pprint
pprint.pprint(country_revenue)

{'Afghanistan': 7819032.05,
 'Albania': 2945119.52,
 'Algeria': 55401.54,
 'Andorra': 6059368.199999999,
 'Angola': 5549949.82,
 'Antigua and Barbuda ': 1357081.78,
 'Armenia': 191709.16999999998,
 'Australia': 5199088.3100000005,
 'Azerbaijan': 1428752.1199999999,
 'Bahrain': 363738.5,
 'Bangladesh': 1146965.36,
 'Barbados': 711214.4600000001,
 'Belarus': 3654833.07,
 'Belgium': 1141909.35,
 'Belize': 16492580.69,
 'Benin': 10864269.61,
 'Bhutan': 5143785.3,
 'Bosnia and Herzegovina': 2292584.95,
 'Botswana': 1386599.6,
 'Bulgaria': 9353285.2,
 'Burkina Faso': 5075869.49,
 'Burundi': 11232361.469999999,
 'Cambodia': 3052016.91,
 'Cameroon': 1367529.5999999999,
 'Canada': 2827749.89,
 'Cape Verde': 465682.89999999997,
 'Central African Republic': 2767841.4899999998,
 'Chad': 163888.17,
 'China': 7675524.680000001,
 'Comoros': 1298587.2400000002,
 'Costa Rica': 5268171.8,
 "Cote d'Ivoire": 2831622.79,
 'Croatia': 12239482.23,
 'Cuba': 1927615.41,
 'Cyprus': 3233638.98,
 'Czech Republic'

## Step 11: Serialization Checkpoint

Save cleaned data to JSON and Parquet.

In [23]:
import pyarrow as pa
import pyarrow.parquet as pq

# Convert to DataFrame for serialization
cleaned_df = pd.DataFrame([t.__dict__ for t in transactions])
cleaned_df.to_json('100000 Sales Records.csv', orient='records', lines=True)

table = pa.Table.from_pandas(cleaned_df)
pq.write_table(table, 'cleaned_transactions.parquet')

## Step 12: Soft Interview Reflection

Using OOP allowed me to encapsulate both data and cleaning logic in a single, reusable structure. This made it easy to apply transformations and feature engineering consistently across all records, improving maintainability and readability.

## Data Dictionary

| Field         | Type    | Description                                 | Source         |
|---------------|---------|---------------------------------------------|----------------|
| region        | string  | Geographical region                         | sales CSV      |
| country       | string  | Country of transaction                      | sales CSV      |
| item_type     | string  | Type of item sold                           | sales CSV      |
| sales_channel | string  | Online or offline sales                     | sales CSV      |
| order_priority| string  | Order priority code                         | sales CSV      |
| order_date    | date    | Date of order                               | sales CSV      |
| order_id      | string  | Unique order identifier                     | sales CSV      |
| ship_date     | date    | Date order was shipped                      | sales CSV      |
| units_sold    | float   | Number of units sold                        | sales CSV      |
| unit_price    | float   | Price per unit                              | sales CSV      |
| unit_cost     | float   | Cost per unit                               | sales CSV      |
| total_revenue | float   | Total revenue for the order                 | sales CSV      |
| total_cost    | float   | Total cost for the order                    | sales CSV      |
| total_profit  | float   | Total profit for the order                  | sales CSV      |
| days_to_ship  | int     | Days between order and shipping             | engineered     |